## Hands-on Tutorial
## Domain-Driven LLM Development: Insights into RAG and Fine-Tuning Practices
### Lab 2.2 : LLM Fine-Tuning through QLoRA.    
#### Summary: 
This lab focused on Instruction fine-tuning a Meta-Llama-3-8B-Instruct model using CUAD data 

- The training dataset is from CUAD - BONTONSTORESINC_04_20_2018-EX-99.3-AGENCY AGREEMENT.PDF  
- The training is transformers trainer through QLoRA     


### Initalization

Uncomment the following lines if the packages are not installed 

In [1]:
#!pip uninstall -y transformers
#!pip install transformers==4.45.2

#!pip install "trl<0.9.0"
#!pip install sentence_transformers
#!pip install datasets

#!pip install accelerate
#!pip install bitsandbytes
#!pip install peft
#!pip install sentence_transformers

#!pip install continuous_eval
#!pip install tiktoken

In [2]:
!nvidia-smi

Sun Sep  8 19:55:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      5MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM

In [4]:
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np

import json

In [ ]:
import nltk
nltk.download('punkt_tab') 

### Load training data

In [5]:
TRN_FILE = '../lab-data/ENERGOUSCORP_qa.csv'
df_train_data = pd.read_csv(TRN_FILE)

sample = Dataset.from_pandas(df_train_data)
sample

Dataset({
    features: ['index', 'question', 'input', 'answer', 'qa_id'],
    num_rows: 32
})

### Prepare for training

In [6]:
# Initialize static strings for the prompt template
INTRO_BLURB = 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n'

INSTRUCTION_KEY = """
[Instruction]: You are a legal AI assistant reviwing commercial contracts. 
Please provide answer to the question listed below about the important contract clauses. 
The questions are provided after the [Question] tag, present your answer after the [Response] tag. 
DO NOT put any premables in the response. If you don't know the answer, just say I don't know, DO NOT make up the answers' 
"""

INPUT_KEY = '[Question]: '
RESPONSE_KEY = '[Response]: '
END_KEY = "[End]"

In [7]:
def create_prompt_formats(sample):

    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{INPUT_KEY}{sample['question']}\n" if sample['question'] else None
    response = f"{RESPONSE_KEY}{sample['answer']}\n"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n".join(parts)

    sample["text"] = formatted_prompt
    
    return sample

In [8]:
from random import randrange

sample_p = create_prompt_formats(sample[randrange(9)])
print(sample_p['text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request. 


[Instruction]: You are a legal AI assistant reviwing commercial contracts. 
Please provide answer to the question listed below about the important contract clauses. 
The questions are provided after the [Question] tag, present your answer after the [Response] tag. 
DO NOT put any premables in the response. If you don't know the answer, just say I don't know, DO NOT make up the answers' 

[Question]: Is there a restriction on the ability of a party to compete with the counterparty or operate in a certain geography or business or technology sector?

[Response]: Until expiration or earlier termination of the Agreement, DIALOG agrees that it and its Affiliates will not, without ENERGOUS' written approval, intentionally sell, distribute or work with any third party to develop products incorporating any Uncoupled Power Transfer Technology other than Licensed Products; provided, howeve

In [9]:
len((sample_p['text']))

2199

In [10]:
def get_max_length(model):

    conf = model.config

    max_length = None

    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
            
    if not max_length:
        max_length = 8192
        print(f"Using default max length: {max_length}")
        
    return max_length

In [11]:
def preprocess_batch(batch, tokenizer, max_length):

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [12]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):

    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ['index', 'question', 'input', 'answer', 'qa_id', 'text'],
    )

    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    dataset = dataset.shuffle(seed = seed)

    return dataset

### Setup model and tokenizer

In [13]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'

Configuration model quantization

In [14]:
load_in_4bit = True
bnb_4bit_use_double_quant = True
bnb_4bit_quant_type = "nf4"
bnb_4bit_compute_dtype = torch.bfloat16

In [15]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
)

LLama model has gated access, please follow this instruction to obtain your token for the llama model: https://huggingface.co/docs/hub/en/security-tokens 

In [16]:
TOKEN = "your_token"  # Replace "your_token" with the token you obtained

Load model and tokenizer

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = "auto", 
    token = TOKEN,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          token = TOKEN,
                                         )
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Preprocess the data

In [18]:
from functools import partial

seed = 0

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, sample)

Found max lenth: 8192
Preprocessing dataset...


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32 [00:00<?, ? examples/s]

In [19]:
print(preprocessed_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 32
})


### Setup training parameters

Setup LoRA parameters

In [20]:
lora_config = LoraConfig(
    r = 64,    
    lora_alpha = 256,    
    target_modules = ['gate_proj', 'up_proj', 'q_proj', 'v_proj', 'down_proj', 'k_proj', 'o_proj'],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [21]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [22]:
model.gradient_checkpointing_enable()    #reduce memory usage during fine-tuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

print_trainable_parameters(model)

All Parameters: 4,708,372,480 || Trainable Parameters: 167,772,160 || Trainable Parameters %: 3.5632728870252848


Setup training parameters

In [23]:
output_dir = "./ft_model_llama3-8b_instruct_cuad"
per_device_train_batch_size = 1
learning_rate = 1e-5   
warmup_steps = 2  # Linear warmup steps from 0 to learning_rate 
fp16 = True  
epoch = 2             
logging_steps = epoch*2
save_steps = 20000

In [24]:
trainer = Trainer(
    model = model,
    train_dataset = preprocessed_dataset,
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        warmup_steps = warmup_steps,
        num_train_epochs = epoch,
        learning_rate = learning_rate,
        fp16 = fp16,
        logging_steps = logging_steps,
        output_dir = output_dir,
        save_strategy="steps",
        save_steps = save_steps,
    ),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
)

In [25]:
!nvidia-smi

Sun Sep  8 19:58:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   39C    P0              32W /  70W |   9960MiB / 15360MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Launch training and save model

In [26]:
model.config.use_cache = False

do_train = True

print("Training...")

if do_train:
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    print(metrics)

Training...


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
4,2.661100
8,2.534900
12,2.066800
16,1.786000
20,1.312800
24,0.920000
28,1.050800
32,0.937500
36,0.707300
40,0.994900


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66de0278-6c7d2bd052497ab2090ab684;248578be-9190-4542-bfa0-4ba36289ecb9)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


***** train metrics *****
  epoch                    =        2.0
  total_flos               =   853031GF
  train_loss               =     1.2349
  train_runtime            = 0:02:47.51
  train_samples_per_second =      0.382
  train_steps_per_second   =      0.382
{'train_runtime': 167.5105, 'train_samples_per_second': 0.382, 'train_steps_per_second': 0.382, 'total_flos': 915935857016832.0, 'train_loss': 1.234889779239893, 'epoch': 2.0}


In [27]:
!nvidia-smi

Sun Sep  8 20:01:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   54C    P0              35W /  70W |  13420MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
print("Saving last checkpoint of the model...")
trainer.model.save_pretrained(output_dir,
                              token = TOKEN,
                              trust_remote_code=True,
                             )

Saving last checkpoint of the model...


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66de0289-0966bbae2c2ae44c300a50ef;b8593309-f9c7-4ff8-bf17-193801759f91)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


### Test inference 

In [29]:
model_ft = model
tokenizer_ft = tokenizer

In [30]:
import time

def Llama_Infer(prompt):

    st = time.time()
    
    batch = tokenizer_ft(prompt, return_tensors="pt")
    input_ids = batch["input_ids"].cuda()

    with torch.no_grad():
    
        output = model_ft.generate(input_ids, 
                                    max_new_tokens=256,
                                    do_sample=True,
                                    temperature = 0.01,
                                    pad_token_id=tokenizer_ft.eos_token_id,
                                    )[0]       

        response = tokenizer_ft.decode(output)



    et = time.time()
    elapsed_time = et - st
    
    #print("generated_text = ", response)
    full_text = response.split('[Response]:')[1].split('[End]')[0].strip()
    answer = full_text
    
    return answer, elapsed_time


In [31]:
IDX = 7

query = df_train_data['question'][IDX]
gt = df_train_data['answer'][IDX]

In [32]:
query, gt

("Which state/country's law governs the interpretation of the contract?",
 'This Letter of Authorization will be governed by and construed in accordance with the laws of California, excluding its conflict of laws provisions, and be subject to the non-exclusive jurisdiction of the California courts., This Agreement will be governed by and construed in accordance with the laws of the State of California, exclusive of conflict of laws principles.')

In [33]:
blurb = f"{INTRO_BLURB}"
instruction = f"{INSTRUCTION_KEY}"
input_context = f'{INPUT_KEY}{query}\n\n{RESPONSE_KEY}'

prompt = blurb+'\n'+instruction+'\n'+input_context
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request. 


[Instruction]: You are a legal AI assistant reviwing commercial contracts. 
Please provide answer to the question listed below about the important contract clauses. 
The questions are provided after the [Question] tag, present your answer after the [Response] tag. 
DO NOT put any premables in the response. If you don't know the answer, just say I don't know, DO NOT make up the answers' 

[Question]: Which state/country's law governs the interpretation of the contract?

[Response]: 


In [34]:
answer, elapse_time = Llama_Infer(prompt)
print("Question = ", query, "\nAnswer = ", answer, "\nGT = ", gt, "\nElapse time = ", elapse_time)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

Question =  Which state/country's law governs the interpretation of the contract? 
Answer =  The Contract will be governed by and construed in accordance with the laws of the State of California, without giving effect to any principles of conflicts of law that would cause the application of the laws of any jurisdiction other than the State of California. 
GT =  This Letter of Authorization will be governed by and construed in accordance with the laws of California, excluding its conflict of laws provisions, and be subject to the non-exclusive jurisdiction of the California courts., This Agreement will be governed by and construed in accordance with the laws of the State of California, exclusive of conflict of laws principles. 
Elapse time =  396.4849715232849


### Optional: restart the kernel and run batch inference on the fine-tuned model

In [ ]:
# --- restart kernel ---
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

In [ ]:
import torch
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM

from sentence_transformers import SentenceTransformer, util

In [ ]:
from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np

import json

Preparation

In [ ]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [ ]:
output_dir = "./ft_model_llama3-8b_instruct_cuad"

In [ ]:
# Initialize static strings for the prompt template
INTRO_BLURB = 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n'

INSTRUCTION_KEY = """
[Instruction]: You are a legal AI assistant reviwing commercial contracts. 
Please provide answer to the question listed below about the important contract clauses. 
The questions are provided after the [Question] tag, present your answer after the [Response] tag. 
DO NOT put any premables in the response. If you don't know the answer, just say I don't know, DO NOT make up the answers' 
"""

INPUT_KEY = '[Question]: '
RESPONSE_KEY = '[Response]: '
END_KEY = "[End]"

Load FM and Peft-load adapter then merge model

In [ ]:
load_in_4bit = True
bnb_4bit_use_double_quant = True
bnb_4bit_quant_type = "nf4"
bnb_4bit_compute_dtype = torch.bfloat16

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
)

In [ ]:
TOKEN = "your_token"  # Replace "your_token" with the token you obtained

In [ ]:
from peft import PeftModel, PeftConfig

model_ft = AutoModelForCausalLM.from_pretrained(  
    model_name,
    quantization_config = bnb_config,
    return_dict=True,
    low_cpu_mem_usage=True,
    device_map="auto",
)

In [ ]:
model_ft = PeftModel.from_pretrained(
    model_ft, 
    output_dir, 
    torch_dtype = torch.float16,
    device_map="auto",
)

In [ ]:
model_ft = model_ft.merge_and_unload()

Load tokenizer

In [ ]:
tokenizer_ft = AutoTokenizer.from_pretrained(model_name)
tokenizer_ft.pad_token = tokenizer_ft.eos_token
tokenizer_ft.padding_side = "right"

In [ ]:
!nvidia-smi

Prepare for model inference and evaluation

In [ ]:
import time

def Llama_Infer(prompt):

    st = time.time()
    
    batch = tokenizer_ft(prompt, return_tensors="pt")
    input_ids = batch["input_ids"].cuda()

    with torch.no_grad():
    
        output = model_ft.generate(input_ids, 
                                    max_new_tokens=256,
                                    do_sample=True,
                                    temperature = 0.01,
                                    pad_token_id=tokenizer_ft.eos_token_id,
                                    )[0]       

        response = tokenizer_ft.decode(output)



    et = time.time()
    elapsed_time = et - st
    
    #print("generated_text = ", response)
    if('[Response]:' in response):
        full_text = response.split('[Response]:')[1].strip()
        if ('[End]' in response):
            full_text = full_text.split('[End]')[0].strip()
    else:
        full_text = response
    answer = full_text
    
    return answer, elapsed_time


In [ ]:
import boto3
import json

boto3_bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")
boto3_bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

def get_titan_embedding(text):
    
    body = json.dumps({"inputText": text})
    modelId = 'amazon.titan-embed-text-v2:0'     
    accept = 'application/json'
    contentType = 'application/json'    
    
    response = boto3_bedrock_runtime.invoke_model(body=body, 
                                                  modelId=modelId, 
                                                  accept=accept, 
                                                  contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    
    return embedding
    
def calculate_semantic_sim_titan(pred_list,ref_list):
   
    sem_score_titan = []
    average_sem_sim = 0
    
    for i in range(len(ref_list)):
        print(i,end = '|')
        ref_embedding = get_titan_embedding(ref_list[i])
        pred_embedding = get_titan_embedding(pred_list[i])
        cos_sim = util.cos_sim(ref_embedding, pred_embedding)
        
        sem_score_titan.append(cos_sim[0][0].item())
    
    return sem_score_titan

In [ ]:
from continuous_eval.metrics.generation.text import DeterministicAnswerCorrectness

def calculate_answer_correctness(pred_list,ref_list):
   
    token_overlap_recall = []
    rouge_l_recall = []
    
    metric = DeterministicAnswerCorrectness()
    
    for i in range(len(ref_list)):
        print(i,end = '|')
    
        datum = {
            "answer": pred_list[i],
            "ground_truth_answers": [ref_list[i]],
        } 
        ac = metric(**datum)    
        
        token_overlap_recall.append(ac['token_overlap_recall'])
        rouge_l_recall.append(ac['rouge_l_recall'])
        
    return token_overlap_recall, rouge_l_recall

Test single inference

In [ ]:
TRN_FILE = '../lab-data/ENERGOUSCORP_qa.csv'
df_test_data = pd.read_csv(TRN_FILE)

In [ ]:
IDX = 10

query = df_test_data['question'][IDX]
gt = df_test_data['answer'][IDX]

blurb = f"{INTRO_BLURB}"
instruction = f"{INSTRUCTION_KEY}"
input_context = f'{INPUT_KEY}{query}\n\n{RESPONSE_KEY}'

prompt = blurb+'\n'+instruction+'\n'+input_context

answer, elapse_time = Llama_Infer(prompt)
print("Question = ", query, "\nAnswer = ", answer, "\nGT = ", gt, "\nElapse time = ", elapse_time)

Batch inference (training data)

In [ ]:
TRN_FILE = '../lab-data/ENERGOUSCORP_qa.csv'
df_test_data = pd.read_csv(TRN_FILE)

In [ ]:
test_question_list = []
test_answer_list = []
test_ref_answer_list = []

st = time.time()

for i in range(len(df_test_data['question'])):
    print(i,end='|')
    
    query = df_test_data['question'][i].strip()
    ref_answer = df_test_data['answer'][i].strip()
    
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f'{INPUT_KEY}{query}\n\n{RESPONSE_KEY}'

    prompt = blurb+'\n'+instruction+'\n'+input_context

    response_text,response_time = Llama_Infer(prompt)
    print(response_text)
    
    test_question_list.append(query)
    test_answer_list.append(response_text)
    test_ref_answer_list.append(ref_answer)
    
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
test_ss_list = calculate_semantic_sim_titan(test_answer_list,test_ref_answer_list)
test_tor_list, test_rlr_list_list = calculate_answer_correctness(test_answer_list,test_ref_answer_list)

average_sem_sim_titan = np.average(test_ss_list)   
average_sem_sim_titan

In [ ]:
df_response = pd.DataFrame()  

df_response["question"] = test_question_list
df_response["ref_answer"] = test_ref_answer_list
df_response["response"] = test_answer_list
df_response["semantic_similarity"] = test_ss_list
df_response["token_overlap_recall"] = test_tor_list
df_response["rouge_l_recall"] = test_rlr_list_list

In [ ]:
df_response

In [ ]:
TEST_OUTPUT_FILE = '../lab-data/sft_trn_q4b_result.csv'
df_response.to_csv(TEST_OUTPUT_FILE, index=False)

Batch inference (testing data)

In [ ]:
TEST_FILE = '../lab-data/ENERGOUSCORP_qa_test.csv'
df_test_data = pd.read_csv(TEST_FILE)

In [ ]:
test_question_list = []
test_answer_list = []
test_ref_answer_list = []

st = time.time()

for i in range(len(df_test_data['question'])):
    print(i,end='|')
    
    query = df_test_data['question'][i].strip()
    ref_answer = df_test_data['answer'][i].strip()
    
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f'{INPUT_KEY}{query}\n\n{RESPONSE_KEY}'

    prompt = blurb+'\n'+instruction+'\n'+input_context

    response_text,response_time = Llama_Infer(prompt)
    print(response_text)
    
    test_question_list.append(query)
    test_answer_list.append(response_text)
    test_ref_answer_list.append(ref_answer)
    
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
test_ss_list = calculate_semantic_sim_titan(test_answer_list,test_ref_answer_list)
test_tor_list, test_rlr_list_list = calculate_answer_correctness(test_answer_list,test_ref_answer_list)

average_sem_sim_titan = np.average(test_ss_list)   
average_sem_sim_titan

In [ ]:
df_response = pd.DataFrame()  

df_response["question"] = test_question_list
df_response["ref_answer"] = test_ref_answer_list
df_response["response"] = test_answer_list
df_response["semantic_similarity"] = test_ss_list
df_response["token_overlap_recall"] = test_tor_list
df_response["rouge_l_recall"] = test_rlr_list_list

In [ ]:
df_response

In [ ]:
TEST_OUTPUT_FILE = '../lab-data/sft_test_q4b_result.csv'
df_response.to_csv(TEST_OUTPUT_FILE, index=False)